In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.distance import great_circle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, cross_val_predict, LeaveOneOut
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, make_scorer

In [2]:
data = pd.read_csv('../data/GHCND_data.csv')

data['DATE'] = pd.to_datetime(data['DATE'])

columns_to_drop = ['TAVG'] 
data = data.drop(columns=columns_to_drop)

data = data.sort_values(by=['STATION', 'DATE'])

# Extract month from the DATE
data['MONTH'] = data['DATE'].dt.month

# Seasonal Features
data['MONTH_SIN'] = np.sin(2 * np.pi * data['MONTH'] / 12)
data['MONTH_COS'] = np.cos(2 * np.pi * data['MONTH'] / 12)

## Creating 3 day lags of PRCP, TMAX and TMIN

In [3]:
# Create lag features for the current station
for lag in range(1, 3):
    data[f'PRCP_LAG_{lag}'] = data.groupby('STATION')['PRCP'].shift(lag)
    data[f'TMAX_LAG_{lag}'] = data.groupby('STATION')['TMAX'].shift(lag)
    data[f'TMIN_LAG_{lag}'] = data.groupby('STATION')['TMIN'].shift(lag)

## Finding the nearest 3 stations for each station

In [4]:
# Get unique stations with their coordinates
stations = data[['STATION', 'LATITUDE', 'LONGITUDE']].drop_duplicates()

# Function to calculate the distance between two coordinates
def calculate_distance(coord1, coord2):
    return great_circle(coord1, coord2).km

# Create a dictionary to store the three closest neighbors for each station
station_nearby = {}

# Calculate distances and store the three closest neighbors for each station
for i, row1 in stations.iterrows():
    distances = []
    for j, row2 in stations.iterrows():
        if i != j:
            coord1 = (row1['LATITUDE'], row1['LONGITUDE'])
            coord2 = (row2['LATITUDE'], row2['LONGITUDE'])
            distance = calculate_distance(coord1, coord2)
            distances.append((row2['STATION'], distance))
    # Sort by distance and take the three closest neighbors
    distances = sorted(distances, key=lambda x: x[1])[:3]
    station_nearby[row1['STATION']] = [neighbor[0] for neighbor in distances]

## Now adding 3 day lags of precipiatationa and temperature of these nearest 3 stations for each station

In [5]:
neighbor_lag_features = pd.DataFrame()

# Create lag features for the three closest neighbors
for station, nearby_stations in station_nearby.items():
    station_data = data[data['STATION'] == station].copy()
    for i, nearby_station in enumerate(nearby_stations):
        for lag in range(1, 3):
            # Create lagged features for the neighboring station
            lagged_cols = data[data['STATION'] == nearby_station][['DATE', 'PRCP', 'TMAX', 'TMIN']].copy()
            lagged_cols['DATE'] = lagged_cols['DATE'] + pd.Timedelta(days=lag)
            lagged_cols = lagged_cols.rename(columns={'PRCP': f'NEIGHBOR_{i+1}_PRCP_LAG_{lag}',
                                                      'TMAX': f'NEIGHBOR_{i+1}_TMAX_LAG_{lag}',
                                                      'TMIN': f'NEIGHBOR_{i+1}_TMIN_LAG_{lag}'})
            station_data = pd.merge(station_data, lagged_cols, on='DATE', how='left')
    neighbor_lag_features = pd.concat([neighbor_lag_features, station_data], axis=0)

neighbor_lag_features = neighbor_lag_features.loc[:, ~neighbor_lag_features.columns.duplicated()]
neighbor_lag_features = neighbor_lag_features.dropna()

data = neighbor_lag_features

# Evaluation 1: Using fixed train, test dataset

In [6]:
# One-hot encode the STATION column
data = pd.get_dummies(data, columns=['STATION'], drop_first=True)

# Define the test set period
test_period_start = '2023-01-01'

# Split the data into training and test sets based on the DATE
train_data = data[data['DATE'] < test_period_start]
test_data = data[data['DATE'] >= test_period_start]

# Define features and target variable
features = train_data.drop(columns=['NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'PRCP'])
target = train_data['PRCP']

# Define the same for the test data
test_features = test_data.drop(columns=['NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'DATE', 'PRCP'])
test_target = test_data['PRCP']

# Normalize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)
test_features_scaled = scaler.transform(test_features)

In [7]:
# Convert data to sequences for LSTM
def create_sequences(data, target, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        label = target[i+sequence_length]
        sequences.append((seq, label))
    return sequences

sequence_length = 30  # Example sequence length, adjust as needed
train_sequences = create_sequences(features_scaled, target.values, sequence_length)
test_sequences = create_sequences(test_features_scaled, test_target.values, sequence_length)

# Create DataLoader for LSTM
batch_size = 64  # Adjust as needed

train_dataset = TensorDataset(
    torch.tensor([seq for seq, label in train_sequences], dtype=torch.float32),
    torch.tensor([label for seq, label in train_sequences], dtype=torch.float32)
)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(
    torch.tensor([seq for seq, label in test_sequences], dtype=torch.float32),
    torch.tensor([label for seq, label in test_sequences], dtype=torch.float32)
)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


/tmp/ipykernel_503909/954747429.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  torch.tensor([seq for seq, label in train_sequences], dtype=torch.float32),


In [8]:
# Convert data to sequences for LSTM
def create_sequences(data, target, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i+sequence_length]
        label = target[i+sequence_length]
        sequences.append((seq, label))
    return sequences

sequence_length = 30  # Example sequence length, adjust as needed
train_sequences = create_sequences(features_scaled, target.values, sequence_length)
test_sequences = create_sequences(test_features_scaled, test_target.values, sequence_length)

# Create DataLoader for LSTM
batch_size = 64  # Adjust as needed

train_dataset = TensorDataset(
    torch.tensor([seq for seq, label in train_sequences], dtype=torch.float32),
    torch.tensor([label for seq, label in train_sequences], dtype=torch.float32)
)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(
    torch.tensor([seq for seq, label in test_sequences], dtype=torch.float32),
    torch.tensor([label for seq, label in test_sequences], dtype=torch.float32)
)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
# Define the LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

input_dim = features_scaled.shape[1]
hidden_dim = 64  # Adjust as needed
output_dim = 1
num_layers = 2  # Adjust as needed

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lstm_model = LSTMModel(input_dim, hidden_dim, output_dim, num_layers).to(device)

# Train the LSTM Model
criterion = nn.MSELoss()
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

num_epochs = 50  # Adjust as needed

In [10]:
lstm_model.train()
for epoch in range(num_epochs):
    for sequences, labels in train_loader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        # Forward pass
        outputs = lstm_model(sequences)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

/home/prasad/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/prasad/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([12])) that is different to the input size (torch.Size([12, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/50], Loss: 6.4374
Epoch [2/50], Loss: 12.3474
Epoch [3/50], Loss: 20.5484
Epoch [4/50], Loss: 7.6913
Epoch [5/50], Loss: 1.8755
Epoch [6/50], Loss: 3.0957
Epoch [7/50], Loss: 4.1674
Epoch [8/50], Loss: 5.2920
Epoch [9/50], Loss: 1.7676
Epoch [10/50], Loss: 2.9145
Epoch [11/50], Loss: 2.0051
Epoch [12/50], Loss: 3.4016
Epoch [13/50], Loss: 5.4867
Epoch [14/50], Loss: 4.0051
Epoch [15/50], Loss: 24.6307
Epoch [16/50], Loss: 6.2062
Epoch [17/50], Loss: 17.0922
Epoch [18/50], Loss: 7.2158
Epoch [19/50], Loss: 4.7858
Epoch [20/50], Loss: 2.1628
Epoch [21/50], Loss: 13.4443
Epoch [22/50], Loss: 32.3025
Epoch [23/50], Loss: 1.9311
Epoch [24/50], Loss: 4.2973
Epoch [25/50], Loss: 6.9391
Epoch [26/50], Loss: 21.7714
Epoch [27/50], Loss: 4.8362
Epoch [28/50], Loss: 25.7907
Epoch [29/50], Loss: 90.8811
Epoch [30/50], Loss: 34.8026
Epoch [31/50], Loss: 14.7798
Epoch [32/50], Loss: 6.2255
Epoch [33/50], Loss: 8.2572
Epoch [34/50], Loss: 6.7291
Epoch [35/50], Loss: 6.7018
Epoch [36/50], Los

In [13]:
# Evaluate the models
def evaluate_model(predictions, true_values):
    mae = mean_absolute_error(true_values, predictions)
    mse = mean_squared_error(true_values, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(true_values, predictions)
    return mae, mse, rmse, r2


# Evaluate the LSTM Model
lstm_model.eval()
lstm_predictions = []
lstm_true_labels = []

with torch.no_grad():
    for sequences, labels in test_loader:
        sequences, labels = sequences.to(device), labels.to(device)
        
        outputs = lstm_model(sequences)
        lstm_predictions.extend(outputs.cpu().numpy())
        lstm_true_labels.extend(labels.cpu().numpy())

# Inverse transform the LSTM predictions to the original scale
lstm_predictions_original = np.array(lstm_predictions) * scaler.scale_[features.columns.get_loc('PRCP_LAG_1')] + scaler.mean_[features.columns.get_loc('PRCP_LAG_1')]

# Calculate evaluation metrics for LSTM
lstm_mae = mean_absolute_error(lstm_true_labels, lstm_predictions_original)
lstm_mse = mean_squared_error(lstm_true_labels, lstm_predictions_original)
lstm_rmse = np.sqrt(lstm_mse)
lstm_r2 = r2_score(lstm_true_labels, lstm_predictions_original)

lstm_evaluation = (lstm_mae, lstm_mse, lstm_rmse, lstm_r2)

# Compile the evaluation results into a DataFrame
evaluation_results = pd.DataFrame({
    'Model': ['LSTM'],
    'MAE': [lstm_evaluation[0]],
    'MSE': [lstm_evaluation[1]],
    'RMSE': [lstm_evaluation[2]],
    'R2': [lstm_evaluation[3]]
})

print(evaluation_results)

# Predict the occurrence of precipitation events > 10 mm/day
threshold = 10  # mm/day
lstm_classification = (lstm_predictions_original > threshold).astype(int)

# Adjust true_classification to match the length of LSTM predictions
true_classification_adjusted = (test_target.values[sequence_length:] > threshold).astype(int)

# Evaluate classification metrics
def evaluate_classification(predictions, true_values):
    accuracy = accuracy_score(true_values, predictions)
    precision = precision_score(true_values, predictions)
    recall = recall_score(true_values, predictions)
    f1 = f1_score(true_values, predictions)
    return accuracy, precision, recall, f1

# Perform classification on the cross-validated predictions
classification_threshold = 10  # mm/day

classification_results = []
true_classification = (test_target > threshold).astype(int)


lstm_classification_metrics = evaluate_classification(lstm_classification, true_classification_adjusted)

# Compile the classification evaluation results into a DataFrame
classification_results = pd.DataFrame({
    'Model': ['LSTM'],
    'Accuracy': [lstm_classification_metrics[0]],
    'Precision': [lstm_classification_metrics[1]],
    'Recall': [lstm_classification_metrics[2]],
    'F1 Score': [lstm_classification_metrics[3]]
})

print(classification_results)

  Model       MAE       MSE      RMSE       R2
0  LSTM  5.833561  39.73175  6.303313 -1.19339
  Model  Accuracy  Precision  Recall  F1 Score
0  LSTM   0.93725        0.0     0.0       0.0


/home/prasad/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Evaluation 2: Using Time based Cross-Validation

In [14]:
# Time-based cross-validation
tscv = TimeSeriesSplit(n_splits=5)

regression_cv_predictions = {}
regression_results = []

lstm_predictions = []
lstm_true_values = []

for train_index, test_index in tscv.split(features_scaled):
    X_train, X_test = features_scaled[train_index], features_scaled[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    
    train_sequences = create_sequences(X_train, y_train.values, sequence_length)
    test_sequences = create_sequences(X_test, y_test.values, sequence_length)
    
    train_dataset = TensorDataset(
        torch.tensor([seq for seq, label in train_sequences], dtype=torch.float32),
        torch.tensor([label for seq, label in train_sequences], dtype=torch.float32)
    )
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    test_dataset = TensorDataset(
        torch.tensor([seq for seq, label in test_sequences], dtype=torch.float32),
        torch.tensor([label for seq, label in test_sequences], dtype=torch.float32)
    )
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    lstm_model = LSTMModel(input_dim, hidden_dim, output_dim, num_layers).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)
    
    # Train the LSTM model
    lstm_model.train()
    for epoch in range(num_epochs):
        for sequences, labels in train_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            
            # Forward pass
            outputs = lstm_model(sequences)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    # Evaluate the LSTM model
    lstm_model.eval()
    with torch.no_grad():
        for sequences, labels in test_loader:
            sequences, labels = sequences.to(device), labels.to(device)
            
            outputs = lstm_model(sequences)
            lstm_predictions.extend(outputs.cpu().numpy())
            lstm_true_values.extend(labels.cpu().numpy())

# Inverse transform the LSTM predictions to the original scale
lstm_predictions_original = np.array(lstm_predictions) * scaler.scale_[features.columns.get_loc('PRCP_LAG_1')] + scaler.mean_[features.columns.get_loc('PRCP_LAG_1')]

# Evaluate LSTM regression metrics
lstm_mae = mean_absolute_error(lstm_true_values, lstm_predictions_original)
lstm_mse = mean_squared_error(lstm_true_values, lstm_predictions_original)
lstm_rmse = np.sqrt(lstm_mse)
lstm_r2 = r2_score(lstm_true_values, lstm_predictions_original)

regression_results.append({
    'Model': 'LSTM',
    'MAE': lstm_mae,
    'MSE': lstm_mse,
    'RMSE': lstm_rmse,
    'R2': lstm_r2
})

regression_results_df = pd.DataFrame(regression_results)
print("Regression Results:")
print(regression_results_df)

/home/prasad/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/prasad/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([23])) that is different to the input size (torch.Size([23, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/prasad/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
 

Regression Results:
  Model       MAE        MSE      RMSE        R2
0  LSTM  5.867324  40.318371  6.349675 -2.053693


In [15]:
# Perform classification on the cross-validated predictions
classification_threshold = 10  # mm/day

classification_results = []

for model_name, predictions_original in regression_cv_predictions.items():
    true_classification_adjusted = (np.array(true_values[:len(predictions_original)]) > classification_threshold).astype(int)
    classification_predictions = (predictions_original > classification_threshold).astype(int)

    # Ensure lengths are consistent
    assert len(true_classification_adjusted) == len(classification_predictions), f"Length mismatch for {model_name}: {len(true_classification_adjusted)} != {len(classification_predictions)}"

    # Evaluate classification metrics
    accuracy = accuracy_score(true_classification_adjusted, classification_predictions)
    precision = precision_score(true_classification_adjusted, classification_predictions)
    recall = recall_score(true_classification_adjusted, classification_predictions)
    f1 = f1_score(true_classification_adjusted, classification_predictions)
    
    classification_results.append({
        'Model': model_name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    })

# Adjust true_classification for LSTM predictions
true_classification_adjusted_lstm = (np.array(lstm_true_values) > classification_threshold).astype(int)
lstm_classification_predictions = (lstm_predictions_original > classification_threshold).astype(int)

# Ensure lengths are consistent
assert len(true_classification_adjusted_lstm) == len(lstm_classification_predictions), f"Length mismatch for LSTM: {len(true_classification_adjusted_lstm)} != {len(lstm_classification_predictions)}"

# Evaluate LSTM classification metrics
lstm_accuracy = accuracy_score(true_classification_adjusted_lstm, lstm_classification_predictions)
lstm_precision = precision_score(true_classification_adjusted_lstm, lstm_classification_predictions)
lstm_recall = recall_score(true_classification_adjusted_lstm, lstm_classification_predictions)
lstm_f1 = f1_score(true_classification_adjusted_lstm, lstm_classification_predictions)

classification_results.append({
    'Model': 'LSTM',
    'Accuracy': lstm_accuracy,
    'Precision': lstm_precision,
    'Recall': lstm_recall,
    'F1 Score': lstm_f1
})

classification_results_df = pd.DataFrame(classification_results)
print("Classification Results:")
print(classification_results_df)


Classification Results:
  Model  Accuracy  Precision  Recall  F1 Score
0  LSTM  0.969192        0.0     0.0       0.0


/home/prasad/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
